# Nonlinear analysis - Assignment 2

04.10.2023 - Jérémie Engler and Laure Toullier

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
import math

np.set_printoptions(precision=5, suppress=True)

Params = plt.rcParams
Params['figure.figsize'] = (14, 7) 

### Part b

#### Geometry

In [2]:
#Geometric data for the truss:
n=3  #nb of nodes

support=[0,2] #number of the nodes where there is a support
act_P=[1]
rests=[1,1,0,0,1,1] #restraints 
DOF=2*n #degree of freedom

a=3000 #mm
b=4000 #mm

In [3]:
#Material Stiffness
E =np.array([200e3, 200e3])  # N/mm^2
sigma_0=np.array([500, 500]) # N/mm^2
eps_0=sigma_0/E
alpha=0.02

In [4]:
#Load definition
load=8675*10**3 #N, value of the applied load
P=np.zeros(DOF).reshape(n,2) 
theta_P=60 #°, inclination of the load regarding the x-axis

    #Application of the load at the corresponding nodes:

P[1]=[load*np.cos(np.radians(theta_P)),load*np.sin(np.radians(theta_P))]
print(P)

[[      0.            0.     ]
 [4337500.      7512770.37783]
 [      0.            0.     ]]


In [5]:
#Generate the nodes of the truss and their location (if 1st node at (0,0))

nodes=np.zeros(n*2).reshape(n,2)

for i in range(n):
    nodes[i,0]=a*i
    if i%2==0:
        nodes[i,1]=0
    else:
        nodes[i,1]=b

print(nodes)

[[   0.    0.]
 [3000. 4000.]
 [6000.    0.]]


In [6]:
#Connectivity: connect nodes with bar element

connectivity=[]

for i in range(n):
    if i<(n-2):
        connectivity.append([i+1, i+2])
#        connectivity.append([i+1, i+3])
    if i==n-2:
        connectivity.append([i+1, i+2])
        
Connectivity=np.array(connectivity)

nb_members=int(np.size(Connectivity)/2)
    
print(Connectivity)

[[1 2]
 [2 3]]


In [7]:
# Area: area of each bar of the truss

#A=10*10**3 #mm²     #members area equal in the entire truss

#area=np.zeros(nb_members)

#for i in range(np.size(area)):
#    area[i]=A
    
area=np.array([15000, 2000])

#### Stiffness

In [8]:
def stiffness (E, area, nodes, nb_members, DOF, Connectivity):


    # Unit stiffness matrix of bar in local coordinate system
    K_unit = np.array([[1, 0, -1, 0], 
                   [0, 0,  0, 0], 
                   [-1, 0, 1, 0], 
                   [0, 0, 0, 0]])
    
    #angles and length
    angles=[]
    length=[]
    
    for i in range(nb_members):
        
        x1=nodes[Connectivity[i,0]-1,0]      #For each node at the extremity of each bar, we get its location in x and y.
        y1=nodes[Connectivity[i,0]-1,1]
        x2=nodes[Connectivity[i,1]-1,0]
        y2=nodes[Connectivity[i,1]-1,1]
        dx=x2-x1
        dy=y2-y1
        length.append(np.sqrt(dx**2+dy**2))
        
        if np.isclose(y1, y2):
            if np.isclose(x1, x2):
                angles.append(90)
            else:
                angles.append(0)
        else:
            theta=np.arcsin((y2-y1)/np.sqrt(dx**2+dy**2)) #calculate the angle between the x-axis and the bar element
            angles.append(math.degrees(theta))
       
    #local stiffness matrix for each bar
    k_local=[]
    
    for i in range(nb_members):
        k_local.append(E[i]*area[i]/length[i])

    #Transformation matrix for each bar element of the truss
    c=[]
    s=[]
    for i in range(nb_members):
        c.append(np.cos(np.radians(angles[i])))
        s.append(np.sin(np.radians(angles[i])))
    
    T=np.zeros((nb_members,4,4))

    for i in range(nb_members):
        T[i,0,0]=c[i]
        T[i,0,1]=s[i]
        T[i,1,0]=-s[i]
        T[i,1,1]=c[i]
        T[i,2,2]=c[i]
        T[i,2,3]=s[i]
        T[i,3,2]=-s[i]
        T[i,3,3]=c[i]
        
    #Stiffness for each bar in global coordinates system
    k_global=[]

    for i in range(nb_members):
        k=k_local[i]*T[i].T@K_unit@T[i] 
        k_global.append(k)

    #Stiffness in global system
    K_globalCS_bar=np.zeros((nb_members, DOF,DOF))
    K_global=np.zeros((DOF,DOF))
    k_global = np.array(k_global)

    for i, ((u, v), k) in enumerate(zip(Connectivity, k_global)):       
        #we build all the global stiffness matrix for each bar in the entire system and sum them.
        K_global[2*u-2:2*u,2*u-2:2*u] += k[0:2, 0:2]
        K_global[2*u-2:2*u,2*v-2:2*v] += k[0:2, 2:4]
        K_global[2*v-2:2*v,2*u-2:2*u] += k[2:4, 0:2]
        K_global[2*v-2:2*v,2*v-2:2*v] += k[2:4, 2:4]
    
        #global stiffness matrix for each bar in the entire system 
        K_globalCS_bar[i, 2*u-2:2*u,2*u-2:2*u] = k[0:2, 0:2]
        K_globalCS_bar[i, 2*u-2:2*u,2*v-2:2*v] = k[0:2, 2:4]
        K_globalCS_bar[i, 2*v-2:2*v,2*u-2:2*u] = k[2:4, 0:2]
        K_globalCS_bar[i, 2*v-2:2*v,2*v-2:2*v] = k[2:4, 2:4]
        
    return (k_local, K_global, K_globalCS_bar, T, length)

#### Equilibrium of the entire system : K*U=R

In [9]:
#Calculate the displacements of the nodes with a load applied

def displacement(support, act_P, P, K_global, DOF):
    
    F_red=np.zeros(DOF-2*len(support))

    for i in range(n):
        if i in act_P:
            F_red[i*2-2]=P[i,0]
            F_red[i*2-1]=P[i,1]  
        
    a=2*(n-len(support)) 
    K_red=np.zeros((a,a)) #reduced stiffness matrix for the nodes where there are displacements
    K_red=K_global[2:a+2, 2:a+2]

    u_red=np.linalg.inv(K_red)@F_red #reduced vector displacements for the nodes concerned by displacements

    u_vec=np.zeros(n*2) #global vector for displacements
    u_vec[2:n*2-2]=u_red[:]

    return(u_vec) 


In [10]:
#Support reactions

def support_reactions (K_global, u_vec):
    F = K_global @ u_vec #N
    return(F)

In [11]:
#Axial forces in each bar of the truss

def axial_load_0(nb_members, K_globalCS_bar,u_vec):

    F_bar=[]
    F_bar_reduced=np.zeros((nb_members,4))
    F_local_CS=[]
    axial_load=[]

    for i in range(nb_members):
        F_bar.append(K_globalCS_bar[i,:,:]@u_vec)   #forces applied at each extermity of each bar in global coordinates in global system
    F_bar=np.array(F_bar)

    for i in range(nb_members): #forces applied at the extremity of each bar in global coordinates in local system
        index_1=Connectivity[i,0] #value of 1st node of the bar
        index_2=Connectivity[i,1] #value of 2nd node of the bar
        F_bar_reduced[i,0]=F_bar[i,(index_1-1)*2] 
        F_bar_reduced[i,1]=F_bar[i,(index_1)*2-1]
        F_bar_reduced[i,2]=F_bar[i,(index_2-1)*2]
        F_bar_reduced[i,3]=F_bar[i,index_2*2-1]     

    for i in range(nb_members): #forces applied at the extremity of each bar in local coordinates in local system
        F_local_CS.append(T[i]@F_bar_reduced[i])   
    F_local_CS=np.array(F_local_CS)

    for i in range(nb_members): #axial load in each bar in N
        axial_load.append(F_local_CS[i,2])
    
    axial_load=np.array(axial_load)
    return axial_load

def axial_load_1(K_globalCS_bar, Connectivity, u_vec, T, nb_members):
    axialF = []
    for i in range(nb_members):
        F_elei = K_globalCS_bar[i,:,:] @ u_vec
        index_red = 2*Connectivity[i]
        F_elei_red = F_elei[[index_red[0]-2, index_red[0]-1, index_red[1]-2, index_red[1]-1]]
        F_elei_local = T[i] @ F_elei_red
        Fi = F_elei_local[2]
        axialF.append(Fi)
    return axialF


#### Newton Raphson Method

In [12]:
#Initialisation

u=np.zeros((DOF)) #initial nodal displacement
print(u)
k, K_global, K_globalCS_bar, T, L=stiffness(E, area, nodes, nb_members, DOF,Connectivity) #initial stiffness matrix
load_incr=10 #increment for loadstep
iter_lim=20
tol=np.max(area)*np.max(sigma_0)/1e5 #tolerance

[0. 0. 0. 0. 0. 0.]


In [13]:
#strains

def strain(Connectivity, u, L, T, nb_members):
    
    u=u.reshape((n,2))
    #displacements in global coordinates
    u_red=np.zeros(nb_members*4).reshape((nb_members,4))
    for i in range(nb_members):  
        u_red[i, 0]=u[Connectivity[i,0]-1,0]      
        u_red[i, 1]=u[Connectivity[i,0]-1,1]
        u_red[i, 2]=u[Connectivity[i,1]-1,0]
        u_red[i, 3]=u[Connectivity[i,1]-1,1]
    
    #displacements in local coordinates
    u_bar=[]
    for i in range(nb_members):
        u_bar.append(T[i]@u_red[i,:])
    
    u_bar=np.array(u_bar)
    
    #calculate the strain in each bar
    strain=[]
    for i in range(nb_members):  
        strain.append((u_bar[i,2]-u_bar[i,0])/L[i])
        
    strain=np.array(strain)
    
    return strain

In [14]:
#internal force for each displacement

def internal_force(Connectivity, area, E, u, L, T, sigma_0, eps_0, nb_members, DOF, rests):
    
    #internal force in each bar
    Pr=[]
    Pr_CS=np.zeros(n*4).reshape((n,4))
    strains=strain(Connectivity, u, L, T, nb_members)
    for i in range(nb_members):
        a= area[i]*sigma_0[i]/2*((3*strains[i]/eps_0[i])-(strains[i]/eps_0[i])**3)
        Pr.append(a)
        Pr_CS[i,0]=-a
        Pr_CS[i,2]=a

    #internal force at each extremity of a bar in local coordinates
   # Pr_CS=np.zeros((nb_members,4))
   # for i in range(nb_members):
   #     Pr_CS[i,0]=-Pr[i]
   #     Pr_CS[i,1]=0
   #     Pr_CS[i,2]=Pr[i]
   #     Pr_CS[i,3]=0
   #Pr_CS=np.array(Pr_CS)
    #Pr_CS = np.array([np.array([-pr, 0, pr, 0]) for pr in Pr])
    
    #internal force at each extremity of a bar in global coordinates
    Pr_glob=np.zeros((2,4))
    for i in range(nb_members):
        Pr_glob[i,:]=np.transpose(T[i])@Pr_CS[i]
    
    #internal force at node 2 for each bar in global coordinates
    Pr_glob_bar1_2=np.array([Pr_glob[0,2],Pr_glob[0,3]])
    Pr_glob_bar2_2=np.array([Pr_glob[1,0],Pr_glob[1,1]])
    #sum of internal force at node 2 in global coordinates
    sum_Pr_glob_2=Pr_glob_bar1_2+Pr_glob_bar2_2
    
    return Pr, Pr_glob, sum_Pr_glob_2


In [15]:
#Generalisation

steps = 1;
lams =  np.linspace(1,load_incr)/load_incr
P_applied=np.zeros((load_incr, DOF))
R=P_applied #initialization of R 
delta_u=np.zeros((1,DOF)) #initialization of delta_u (no force, no displacement)

#lists to generate later the graphs !
u1=[]
u2=[]
p1=[]
p2=[]
u_print=[]
r=[]

#loops:
for i in range (load_incr):
    #load P is applied by increment
    P_applied[i,:]=P.reshape((1,6))*lams[i]
    #first increment of displacement is calculated with the function displacement (get the displacement of each node for a load applied)
    delta_u=displacement(support, act_P, P_applied[i,:].reshape(3,2), K_global, DOF)
    print(f'delta_u={delta_u}')
    p1.append(P_applied[i,2])
    p2.append(P_applied[i,3])
    
    for j in range(iter_lim): 
        u=u+delta_u
        u_print.append(u) #(1*6)
        print(f'u={u}')
        nodes=nodes+np.array(delta_u).reshape(3,2) #new global displacements 
        print(f'nodes={nodes}')
    
        #as we do have new displacements, we need to calculate new length, new stiffness, n 
        k, K_global, K_globalCS_bar, T, L=stiffness(E, area, nodes, nb_members, DOF,Connectivity)
        print(f'u_after={u}')
        print(f'K_global={K_global}')
        #has the bar yielded ?
        for m in range(nb_members):
            eps=strain(Connectivity, u, L, T, nb_members) #returns an array with the strain of each bar
            print(f'eps={eps}')
            if np.abs(eps[m]) >= eps_0[m]: #the bar has yielded, the Young's modulus of the bar needs to be updated!
                E[m]=alpha*E[m]   
                k, K_global, K_globalCS_bar, T, L=stiffness(E, area, nodes, nb_members, DOF,Connectivity) #the stiffness matrix has changed as E has changed!
            else:
                E[m]=E[m]  #the bar has not reached its yield limit !
        
        #internal force Pr at node 2
        Pr, Pr_glob, sum_Pr_glob_2 = internal_force(Connectivity, area, E, u, L, T, sigma_0, eps_0, nb_members, DOF, rests)
        
        #sum of internal force Pr at node 2 in global coordinates:
        sum_Pr_glob=np.zeros(6)
        sum_Pr_glob[2]=sum_Pr_glob_2[0]
        sum_Pr_glob[3]=sum_Pr_glob_2[1]
        
        #Residual force in global coordinates
        R=P_applied[i,:]-sum_Pr_glob
        r.append(R)
        norm_R=np.linalg.norm(R)       
    
        if norm_R< tol:
            u1.append(u[2])
            u2.append(u[3])
            break #if the tolerance is reached, we need to go "out of the loop" to go to the newt load iteration

        if norm_R>tol and j==iter_lim:
            print("No convergence !")
        
        else:
            #Update the strain-increment delta_u in global coordinates:
            delta_u=np.linalg.inv(K_global)@(R.reshape((DOF,1)))
            delta_u=delta_u.reshape(1,DOF)
            print(f'delta_u={delta_u}')

print(E)
print(alpha)

delta_u=[0.      0.      0.02824 1.71008 0.      0.     ]
u=[0.      0.      0.02824 1.71008 0.      0.     ]
nodes=[[   0.         0.     ]
 [3000.02824 4001.71008]
 [6000.         0.     ]]
u_after=[0.      0.      0.02824 1.71008 0.      0.     ]
K_global=[[ 215824.65218  287886.51898 -215824.65218 -287886.51898       0.
        0.     ]
 [ 287886.51898  384009.18048 -287886.51898 -384009.18048       0.
        0.     ]
 [-215824.65218 -287886.51898  244600.77373  249501.5924   -28776.12155
    38384.92658]
 [-287886.51898 -384009.18048  249501.5924   435211.44505   38384.92658
   -51202.26457]
 [      0.            0.       -28776.12155   38384.92658   28776.12155
   -38384.92658]
 [      0.            0.        38384.92658  -51202.26457  -38384.92658
    51202.26457]]
eps=[0.00028 0.00027]
eps=[0.00028 0.00027]
delta_u=[[-8.07624e+15  4.45104e+16 -1.67335e+16  5.10007e+16 -2.55994e+15
   6.16262e+16]]
u=[[-8.07624e+15  4.45104e+16 -1.67335e+16  5.10007e+16 -2.55994e+15
   6.16262e

LinAlgError: Singular matrix

In [ ]:
def printgraph(x,y,xname,yname,unit1, unit2):
    plt.plot(x,y,'o')
    plt.plot(x,y,'b')

    plt.title(f'{yname} as a function of {xname}')
    plt.xlabel(f"{xname} ({unit1})")
    plt.ylabel(f"{yname} ({unit2})")

    plt.show()

    return None

In [ ]:
printgraph(u1,p1,"u1","p1","mm","N")
printgraph(u2,p2,"u2","p2","mm","N")
printgraph(u1,u2,"u1","u2","mm","mm")
printgraph(u_print,r,"u","R","mm","N")

In [ ]:
#tests
for lam_i in range(np.size(lams)):
    lam = lams[lam_i];
    for i in range(iter_lim):
        P = lam * P
        #displacement
        delta_u=displacement(support, act_P, P_applied, K_global, DOF)
        delta_u=delta_u.reshape((n,2))
        u=u_0+delta_u
        nodes=nodes+np.array(delta_u).reshape(3,2) #new global displacements 
        
        Pr, Pr_glob, sum_Pr_glob_2 = internal_force(Connectivity, area, E, u, L, T, sigma_0, eps_0, nb_members, DOF, rests)
        
        #if we consider P extern as the reaction forces in each bar !
        #P=support_reactions (K_global, u.reshape(DOF,1)) 
        #P_bar=np.array([np.sqrt(P[0]**2+P[1]**2), np.sqrt(P[4]**2+P[5]**2) ]).reshape((1,2)).flatten() 
        
        sum_Pr_glob=np.zeros(6)
        sum_Pr_glob[2]=sum_Pr_glob_2[0]
        sum_Pr_glob[3]=sum_Pr_glob_2[1]
        R=P_applied.reshape((1,6))-sum_Pr_glob
        norm_R=np.linalg.norm(R)       
    
        k, K_global, K_globalCS_bar, T, L=stiffness(E, area, nodes, nb_members, DOF,Connectivity)
        
        #update the strain-increment delta_u:
        delta_u=np.linalg.inv(K_global)@R.reshape((6,1))
        #du = np.linalg.inv(K_globalCS_bar[i,:,:]) * R[i]    #du code de la prof
 
        #has the bar yielded ?
        for i in range(nb_members):
            strains[i] = strain(Connectivity, area, E, u, L, T, nb_members)
            if np.abs(strains(m)) >= eps_0:
                E[i]=alpha*E[i]   #if the yield limit is reached, Young's modulus is updated !
            else:
                E[i]=E[i]